# 반복형, 반복자, 제너레이터

## 단어 시퀀스

In [1]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)  # <1>

    def __getitem__(self, index):
        return self.words[index]  # <2>

    def __len__(self):  # <3>
        return len(self.words)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)  # <4>

In [4]:
s = Sentence('"The time has come", the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [5]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [6]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

## 반복형과 반복자

In [7]:
s = Sentence('The time has come')
it = iter(s)
next(it)
'The'
next(it)
'time'
next(it)
'has'
next(it)
'come'
next(it)

StopIteration: 

In [8]:
list(it)

[]

In [9]:
list(iter(s))

['The', 'time', 'has', 'come']

In [10]:
# 고전적인 반복자
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):  # <1>
        return SentenceIterator(self.words)  # <2>


class SentenceIterator:

    def __init__(self, words):
        self.words = words  # <3>
        self.index = 0  # <4>

    def __next__(self):
        try:
            word = self.words[self.index]  # <5>
        except IndexError:
            raise StopIteration()  # <6>
        self.index += 1  # <7>
        return word  # <8>

    def __iter__(self):  # <9>
        return self
# END SENTENCE_ITER

def main():
    import sys
    import warnings
    try:
        filename = sys.argv[1]
        word_number = int(sys.argv[2])
    except (IndexError, ValueError):
        print('Usage: %s <file-name> <word-number>' % sys.argv[0])
        sys.exit(1)
    with open(filename, 'rt', encoding='utf-8') as text_file:
        s = Sentence(text_file.read())
    for n, word in enumerate(s, 1):
        if n == word_number:
            print(word)
            break
    else:
        warnings.warn('last word is #%d, "%s"' % (n, word))

## 제너레이터 함수

In [ ]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for word in self.words:  # <1>
            yield word  # <2>
        return  # <3>


In [11]:
def gen_AB():
    print('start')
    yield "A"
    print('continue')
    yield "B"
    print('end')

for c in gen_AB():
    print('--->', c)

start
---> A
continue
---> B
end


In [12]:
# 느긋한 구현
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text  # <1>

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for match in RE_WORD.finditer(self.text):  # <2>
            yield match.group()  # <3>

## 제너레이터 표현식

In [13]:
def gen_AB():
    print('start')
    yield "A"
    print('continue')
    yield "B"
    print('end')

res1 = [x*3 for x in gen_AB()]

for i in res1:
    print('--->', i)

res2 = (x*3 for x in gen_AB())
print(res2)
for i in res2:
    print('--->', i)

start
continue
end
---> AAA
---> BBB
<generator object <genexpr> at 0x000001E210186740>
start
---> AAA
continue
---> BBB
end


In [14]:
# 제너레이터 표현식 사용
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

## 등차수열 제너레이터

In [16]:
class ArithmeticProgression:

    def __init__(self, begin, step, end=None):  # <1>
        self.begin = begin
        self.step = step
        self.end = end  # None -> "infinite" series

    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)  # <2>
        forever = self.end is None  # <3>
        index = 0
        while forever or result < self.end:  # <4>
            yield result  # <5>
            index += 1
            result = self.begin + self.step * index  # <6>

In [17]:
ap = ArithmeticProgression(0, 1, 3)
print(list(ap))
ap = ArithmeticProgression(1, .5, 3)
print(list(ap))
ap = ArithmeticProgression(0, 1/3, 1)
print(list(ap))
from fractions import Fraction
ap = ArithmeticProgression(0, Fraction(1, 3), 1)
print(list(ap))
from decimal import Decimal
ap = ArithmeticProgression(0, Decimal('.1'), .3)
print(list(ap))

[0, 1, 2]
[1.0, 1.5, 2.0, 2.5]
[0.0, 0.3333333333333333, 0.6666666666666666]
[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]
[Decimal('0'), Decimal('0.1'), Decimal('0.2')]


In [19]:
# 제너레이터 함수
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index

In [20]:
# itertools 사용
import itertools


def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

In [22]:
gen = itertools.takewhile(lambda n:n<3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

## 표준 라이브러리의 제너레이터 함수

In [24]:
import itertools

for char, group in itertools.groupby('LLLLAAAGG'):
    print(char, '-->', list(group))

L --> ['L', 'L', 'L', 'L']
A --> ['A', 'A', 'A']
G --> ['G', 'G']


## 파이썬 3.3의 새로운 구문 yield from

In [25]:
def chain(*iterable):
    for it in iterable:
        for i in it:
            yield i

s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

In [26]:
def chain(*iterable):
    for i in iterable:
        yield from i # replaces for loop completely
    
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

### yield 키워드는 자신을 내포한 함수만 제너레이터 함수로 만든다.

In [31]:
# 생성과정을 추상화 한 것 처럼 보이는 코드
def f():
    def do_yield(n):
        yield n
    x = 0
    while True:
        x += 1
        do_yield(x)

In [ ]:
g = f()
# 무한 루프에 빠짐
print(next(g))
print(next(g))
print(next(g))

In [33]:
def f():
    def do_yield(n):
        yield n
    x = 0
    while True:
        x += 1
        yield from do_yield(x)

In [34]:
g = f()
print(next(g))
print(next(g))
print(next(g))

1
2
3


## 피보나치 수열을 파이썬스럽게 구현하기

In [35]:
# 제너레이터 객체를 사용하지 않는 제너레이터
class Fibonacci:

    def __iter__(self):
        return FibonacciGenerator()


class FibonacciGenerator:

    def __init__(self):
        self.a = 0
        self.b = 1

    def __next__(self):
        result = self.a
        self.a, self.b = self.b, self.a + self.b
        return result

    def __iter__(self):
        return self

In [36]:
# 파이써닉하게 구현
def fibonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

In [40]:
gen_fibo = fibonacci()
for i in range(10):
    print(next(gen_fibo))

0
1
1
2
3
5
8
13
21
34
